In [1]:
# %%capture
# !pip install unsloth
# # Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
# %pip uninstall unsloth unsloth_zoo -y
# %pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# %pip install --upgrade --no-cache-dir "git+https://github.com/unslothai/unsloth-zoo.git"

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-02-12 10:37:33.631071: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-12 10:37:33.645439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739324253.661728  553444 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739324253.666633  553444 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 10:37:33.684455: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.4: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.0. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

In [ ]:
import json

import time

def load_json(file_paths):
    data = []
    for file_path in file_paths:
        with open(file_path, "r", encoding="utf-8") as file:
            data.extend(json.load(file))
    return data

def save_to_jsonl(file_path, data):
    with open(file_path, "a", encoding="utf-8") as f: 
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

def generate_response(instruction_text, input_text, max_tokens):
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction_text, # instruction
            input_text, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, 
                             max_new_tokens=max_tokens, 
                             use_cache=True,)
    decoded_outputs = tokenizer.batch_decode(outputs)

    response_texts = [output.split("### Response:\n")[-1].strip() for output in decoded_outputs]

    return response_texts[0].replace("<|eot_id|>", "")

# 실행 
input_json_path = input_json_paths = [
    "/data/jaesung/llm_for_diabetes/src/model/mLoRA/demo/data_classification_test.json",
    "/data/jaesung/llm_for_diabetes/src/model/mLoRA/demo/data_generation_test.json"
]
output_json_path = "/data/jaesung/llm_for_diabetes/src/model/unsloth/model_output/zero_shot_model_output.jsonl"

data = load_json(input_json_path)

start_time = time.time()
total_samples = len(data)
for idx, item in enumerate(data):
    sample_start_time = time.time()

    input_text = item.get("input", "")
    instruction = item.get("instruction", "")
    task = item.get("task", "")

    max_tokens = 32 if task in ["qa1", "qa2", "qa3", "nli", "ie", "re"] else 512 

    model_output = generate_response(instruction, input_text, max_tokens)

    output_data = item.copy()
    output_data.update({
        "model_output": model_output,
    })
    save_to_jsonl(output_json_path, output_data)

    elapsed_time = time.time() - start_time
    avg_time_per_sample = elapsed_time / (idx + 1) 
    remaining_samples = total_samples - (idx + 1)
    estimated_remaining_time = remaining_samples * avg_time_per_sample

    print(f"[{idx+1}/{total_samples}] Sample processed in {time.time() - sample_start_time:.2f}s, ETA: {estimated_remaining_time/60:.2f} min")

print(f"\nAll samples processed. Total time: {(time.time() - start_time)/60:.2f} min")

[1/1181] Sample processed in 14.43s, ETA: 283.83 min
[2/1181] Sample processed in 11.04s, ETA: 250.31 min
[3/1181] Sample processed in 22.71s, ETA: 315.37 min
[4/1181] Sample processed in 42.80s, ETA: 446.21 min
[5/1181] Sample processed in 8.58s, ETA: 390.30 min
[6/1181] Sample processed in 42.81s, ETA: 464.69 min
[7/1181] Sample processed in 42.60s, ETA: 517.03 min
[8/1181] Sample processed in 9.15s, ETA: 474.39 min


KeyboardInterrupt: 

In [15]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Analyze the sentence with two drugs labeled as @DRUG_A$ and @DRUG_B$. Extract the interaction between @DRUG_A$ and @DRUG_B$ from the input sentence by selecting only one of the following options: 'DDI-effect', 'DDI-mechanism', 'DDI-advise', 'DDI-false', and 'DDI-int'. 'DDI-effect': Choose this when the interaction describes an effect or a pharmacodynamic mechanism. 'DDI-mechanism': Choose this for interactions explained by pharmacokinetic mechanisms. 'DDI-advise': Choose this when the sentence provides a recommendation or advice about the drug interaction. 'DDI-false': Choose this if there is no actual drug-drug interaction in the sentence. 'DDI-int': Choose this when a drug-drug interaction is mentioned without additional detail.", # instruction
        "@DRUG_A$ did not interfere with the absorption or disposition of the @DRUG_B$ glyburide in diabetic patients.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
decoded_outputs = tokenizer.batch_decode(outputs)

response_texts = [output.split("### Response:\n")[-1].strip() for output in decoded_outputs]

print(response_texts[0]) 


DDI-false<|eot_id|>
